## 5-2: Gradient Boosting Model
### 5-2-1: Gradient Boosting with Holdout test set

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
#from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import numpy as np
from datetime import datetime


pd.set_option('display.max_rows', None, 'display.max_colwidth', 500)


def GBoosting_GridSearch(Lable, n_est, depth, lr, feature_type, is_print = True):
    if Lable == 'IE':
        s_Lable = 'I'
        predict_Lable = 'Introverts'
    elif Lable == 'NS':
        s_Lable = 'N'
        predict_Lable = 'Intuitives'
    elif Lable == 'FT':
        s_Lable = 'F'
        predict_Lable = 'Feelers'
    elif Lable == 'PJ':
        s_Lable = 'P'
        predict_Lable = 'Perceivers'

    gb = GradientBoostingClassifier(n_estimators=n_est, max_depth=depth , learning_rate=lr)
    gb_model = gb.fit(X_train_set, Y_train[Lable])
    Y_pred = gb_model.predict(X_test_set)

    precision, recall, fscore, support = score(Y_test[Lable], Y_pred, pos_label=s_Lable, average='binary')
    if is_print:
        print('Being ' + predict_Lable + ' using ' + feature_type + ': '  ,'Estimators: {} / Max_Depth: {} / Learning_Rate: {} --> Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                        n_est,
                                                        depth,
                                                        lr,
                                                        round(precision, 3),
                                                        round(recall, 3), 
                                                        round((Y_pred==Y_test[Lable]).sum() / len(Y_pred),3)))
    return([predict_Lable, feature_type, n_est, depth, lr, precision, recall, round((Y_pred==Y_test[Lable]).sum() / len(Y_pred),3)])



classes = ['FT']
gb_result = []

Y_train = pd.read_pickle('Y_train.pkl')
Y_test = pd.read_pickle('Y_test.pkl')



for feature_type in ['TFIDF', 'CountVectorizer']:
    if feature_type == 'TFIDF':
        X_test_set = np.load('X_test_tfidf.npy')
        X_train_set = np.load('X_train_tfidf.npy')
    elif feature_type == 'CountVectorizer':
        X_test_set = np.load('X_test_count.npy')
        X_train_set = np.load('X_train_count.npy')
    #elif feature_type == 'N-gram':
        #X_test_set = np.load('X_test_ngram.npy')
        #X_train_set = np.load('X_train_ngram.npy')
        #print()
    for item in classes:
        for n_est in [50, 100, 150]:
            for depth in [3, 7, 11, 15]:
                for lr in [0.01, 0.10, 1.00]:
                    gb_result.append(GBoosting_GridSearch(Lable = item, n_est=n_est, depth=depth, lr = lr, feature_type=feature_type))
                    pd.DataFrame(gb_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Learning_Rate','Precision', 'Recall', 'Accuracy']).to_csv('GB_Holdout_Result.csv')

pd.DataFrame(gb_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Learning_Rate','Precision', 'Recall', 'Accuracy']).to_csv('GB_Holdout_Result.csv')
#pd.DataFrame(gb_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Learning_Rate','Precision', 'Recall', 'Accuracy']).head()


### 5-2-2: Evaluation Gradient Boosting Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', 500)


def GB_Evaluation(param, Lable, is_print = True):
    gb = GradientBoostingClassifier()
    gs = GridSearchCV(gb, param, cv=5)#cv=5 meand 5 folde validation
    gs_fit = gs.fit(X_train_set, Y_train[Lable])

    if is_print:
        print(pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[['param_max_depth', 
        'param_n_estimators', 'param_learning_rate','std_test_score', 'mean_test_score', 'rank_test_score']][0:5])

    return(pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[['param_max_depth',
     'param_n_estimators', 'param_learning_rate', 'std_test_score', 'mean_test_score', 'rank_test_score']][0:5])
  
param = {'n_estimators' : [50, 100, 150],
                'max_depth' : [7, 11, 15],
                'learning_rate' : [0.1,0.25]}

classes = ['IE' , 'NS', 'FT', 'PJ']
Y_train = pd.read_pickle('Y_train.pkl')
gb_parm_result = []


for feature_type in ['TFIDF', 'CountVectorizer']:
    if feature_type == 'TFIDF':
        X_train_set = np.load('X_train_tfidf.npy')
    elif feature_type == 'CountVectorizer':
        X_train_set = np.load('X_train_count.npy')
    #elif feature_type == 'N-gram':
        #X_train_set = np.load('X_train_ngram.npy')
    for item in classes:
        gb_parm_result.append(GB_Evaluation(param = param, Lable = item))

pd.DataFrame(gb_parm_result).to_csv('GB_Parm_Result.csv')

## Final evaluation of models

In [24]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time
import math

Y_train = pd.read_pickle('Y_train.pkl')
Y_test = pd.read_pickle('Y_test.pkl')
X_train_tfidf = np.load('X_train_tfidf.npy')
X_test_tfidf = np.load('X_test_tfidf.npy')

df = pd.read_csv('RF_Parm_Result.csv')
df = df.sort_values(['rank_test_score'], ascending = True).groupby(['Class']).head(1)

def RF_Final_Eval(item):
    if item['Class'] == 'IE':
        Lable, s_Lable, predict_Lable = 'IE', 'I', 'Introverts'
 
    elif item['Class'] == 'NS':
        Lable, s_Lable, predict_Lable = 'NS', 'N', 'Intuitives'

    elif item['Class'] == 'FT':
        Lable, s_Lable, predict_Lable = 'FT','F', 'Feelers'

    elif item['Class'] == 'PJ':
        Lable, s_Lable, predict_Lable = 'PJ','P', 'Perceivers'
    
    if math.isnan(item['param_max_depth']):
        item['param_max_depth'] = None

    rf = RandomForestClassifier(n_estimators=item['param_n_estimators'], max_depth=item['param_max_depth'], n_jobs=-1)

    start = time.time()
    rf_model = rf.fit(X_train_tfidf, Y_train[Lable])
    end = time.time()
    fit_time = (end - start)

    start = time.time()
    Y_pred = rf_model.predict(X_test_tfidf)
    end = time.time()
    pred_time = (end - start)

    precision, recall, fscore, train_support = score(Y_test[Lable], Y_pred, pos_label=s_Lable, average='binary')

    print('{} --> Fit time: {} / Predict time: {} --> Estimator {} / Max_Depth {} --> Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                    predict_Lable,
                                                    round(fit_time, 3),
                                                    round(pred_time, 3),
                                                    item['param_n_estimators'],
                                                    item['param_max_depth'],
                                                    round(precision, 3),
                                                    round(recall, 3), 
                                                    round((Y_pred==Y_test[Lable]).sum() / len(Y_pred),3)))


for index, item in df.iterrows():
    RF_Final_Eval(item)


Introverts --> Fit time: 15.922 / Predict time: 0.686 --> Estimator 10 / Max_Depth 30.0 --> Precision: 0.755 / Recall: 0.997 / Accuracy: 0.754
Feelers --> Fit time: 293.472 / Predict time: 1.353 --> Estimator 500 / Max_Depth None --> Precision: 0.731 / Recall: 0.896 / Accuracy: 0.761
Perceivers --> Fit time: 90.712 / Predict time: 1.005 --> Estimator 100 / Max_Depth None --> Precision: 0.644 / Recall: 0.964 / Accuracy: 0.646
Intuitives --> Fit time: 38.441 / Predict time: 0.698 --> Estimator 10 / Max_Depth None --> Precision: 0.856 / Recall: 0.999 / Accuracy: 0.855


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time
import pandas as pd
import numpy as np

classes = ['IE' , 'NS', 'FT', 'PJ']
Y_train = pd.read_pickle('Y_train.pkl')
Y_test = pd.read_pickle('Y_test.pkl')
X_train_tfidf = np.load('X_train_tfidf.npy')
X_test_tfidf = np.load('X_test_tfidf.npy')

df = pd.read_csv('GB_Parm_Result.csv')
df = df.sort_values(['rank_test_score'], ascending = True).groupby(['Class']).head(1)

for index, item in df.iterrows():
    if item['Class'] == 'IE':
        s_Lable = 'I'
    elif item['Class'] == 'NS':
        s_Lable = 'N'
    elif item['Class'] == 'FT':
        s_Lable = 'F'
    elif item['Class'] == 'PJ':
        s_Lable = 'P'

    
    gb = GradientBoostingClassifier(n_estimators=item['param_n_estimators'], max_depth=item['param_max_depth'], lr=)

    start = time.time()
    gb_model = gb.fit(X_train_tfidf,Y_train[])
    end = time.time()
    fit_time = (end - start)

    start = time.time()
    Y_pred = gb_model.predict(X_test_tfidf)
    end = time.time()
    pred_time = (end - start)


precision, recall, fscore, train_support = score(Y_test[Lable], Y_pred, pos_label=s_Lable, average='binary')

print('Fit time: {} / Predict time: {} --> Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                        round(fit_time, 3),
                                                        round(pred_time, 3),
                                                        round(precision, 3),
                                                        round(recall, 3), 
                                                        round((Y_pred==Y_test[Lable]).sum() / len(Y_pred),3)))
                                                        

In [5]:
import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)